In [1]:
import redis
import json
import uuid
import paho.mqtt.client as mqtt

# Connect to Redis
REDIS_HOST = 'redis-11633.c293.eu-central-1-1.ec2.cloud.redislabs.com' 
REDIS_PORT = 11633
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'ohqqd7LasHtDIbzLOt00SHV393FUhATt'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

# Create a new MQTT client
client = mqtt.Client()

# Define the callbacks for when the client receives a response from the server
# and when a message is published on a subscribed topic
def on_connect(client, userdata, flags, rc):
    print(f'Connected with result code {str(rc)}')
    # Subscribe to a topic when the client connects
    client.subscribe('s316411')

def on_message(client, userdata, msg):
    message = msg.payload.decode()
    topic = msg.topic
    print(f"Received message '{message}' on topic {topic}")

    # Parse the received JSON message
    try:
        data = json.loads(message)
        events = data.get('events', [])
        mac_address = data.get("mac_address")
        if not redis_client.exists(f'{mac_address}:battery') or not redis_client.exists(f'{mac_address}:power'):
             redis_client.ts().create(f'{mac_address}:battery')
             redis_client.ts().create(f'{mac_address}:power')
        
        if events:
            for event in events:
                timestamp = event.get('timestamp')
                battery_level = event.get('battery_level')
                power_plugged = event.get('power_plugged')

                # Store data in Redis time series
                redis_client.ts().add(f'{mac_address}:battery', timestamp, battery_level)
                redis_client.ts().add(f'{mac_address}:power', timestamp, power_plugged)

                print(f"Data stored in Redis for {mac_address}: battery={battery_level}, power={power_plugged}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    except KeyError as e:
        print(f"KeyError: {e}")

# Set the callbacks
client.on_connect = on_connect
client.on_message = on_message

client.connect('mqtt.eclipseprojects.io', 1883)

# Start the client loop to process network events and call the callbacks
client.loop_forever()


Redis Connected: True
Connected with result code 0
Received message '{
    "mac_address": "0xf14d2b988308",
    "events": [
        {
            "timestamp": 1706799565249,
            "battery_level": 62,
            "power_plugged": 0
        },
        {
            "timestamp": 1706799566255,
            "battery_level": 62,
            "power_plugged": 0
        },
        {
            "timestamp": 1706799567270,
            "battery_level": 62,
            "power_plugged": 0
        },
        {
            "timestamp": 1706799568273,
            "battery_level": 62,
            "power_plugged": 0
        },
        {
            "timestamp": 1706799569274,
            "battery_level": 62,
            "power_plugged": 0
        },
        {
            "timestamp": 1706799570281,
            "battery_level": 62,
            "power_plugged": 0
        },
        {
            "timestamp": 1706799571286,
            "battery_level": 62,
            "power_plugged": 0
        },
 

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3de268ea-71d7-469a-8d76-c287e5ad6570' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>